In [ ]:
from TTS.api import TTS
tts = TTS('tts_models/multilingual/multi-dataset/xtts_v2')

In [ ]:
import pyaudio
import numpy as np

VOLUME_ADJUSTMENT = 0.05

p = pyaudio.PyAudio()
OUT_STREAM = p.open(rate=22050, channels=1, format=pyaudio.paFloat32, output=True)

def push_speech_to_audio_stream(phrase:str):
    wf = tts.tts(
        text = phrase,
        language = 'en',
        speaker_wav = 'hagrid_voice_trimmed.wav',
        split_sentences = True,
        speed = 2.0,
    )
    buf = np.asarray(wf, dtype=np.float32) * VOLUME_ADJUSTMENT
    OUT_STREAM.write(buf.tobytes())
    print(phrase)

In [ ]:
import re
import threading
import ollama

model_id = 'hagrid'
if model_id in ollama.list():
    ollama.delete(model_id)
ollama.create(model_id, path=f"{model_id}.modelfile")

phrase_end_pattern = re.compile(r'[!\?\.\,]')
action_pattern = re.compile(r'\*\w+\*')

phrase = []
threads = []
for part in ollama.generate(model_id, prompt='hello', stream=True):
    token = part['response']
    phrase.append(token)
    token = action_pattern.sub('', token)
    if phrase_end_pattern.search(token):
        output = ''.join(phrase)
        t = threading.Thread(target=push_speech_to_audio_stream, name='TTS', kwargs={'phrase':output})
        t.start()
        threads.append(t)
        phrase = []

OUT_STREAM.start_stream()
for t in threads:
    t.join()